In [9]:
import itertools
import numpy as np
import pandas as pd
import matplotlib
import seaborn as sns
import plotnine
from plotnine import *
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer

from captum.attr import visualization as viz
from captum.attr import LayerIntegratedGradients
from captum.attr import configure_interpretable_embedding_layer, remove_interpretable_embedding_layer

plotnine.options.figure_size = (12, 12)
import warnings
warnings.filterwarnings("ignore")

In [2]:
transformer = "distilbert-base-cased"
#transformer = "roberta-base"
#transformer = "twmkn9/bert-base-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(transformer)
model = AutoModelForMaskedLM.from_pretrained(transformer)
model.eval()
model.zero_grad()

In [3]:
def prepare_data_for_sentence(sent, N):
    input_seq = tokenizer.encode(sent)

    mask_index = input_seq.index(tokenizer.mask_token_id)
#    print(mask_index)

    input_tensor = torch.tensor([input_seq])

    input_result = model.forward(input_tensor, return_dict=True)

    input_result.logits.shape

    token_logits = input_result.logits
    mask_token_logits = token_logits[0, mask_index, :]
    mask_token_probs = torch.nn.functional.softmax(mask_token_logits, dim=0)

    # get the top predictions for the non-occluded sentence
    top_N = torch.topk(mask_token_probs, N, dim=0)
#    print(top_N)
    probs = top_N.values.tolist()
    top_N_tokens = top_N.indices.tolist()
#    print(probs, sum(probs), top_N_tokens)
    return input_seq, top_N_tokens, mask_index

In [4]:
def custom_forward(inputs, attention_mask=None, pos=0):
#    result = model.forward(inputs.double(), return_dict=True, attention_mask=attention_mask)
#    print("POS", pos)
    result = model.forward(inputs, return_dict=True, attention_mask=attention_mask)
    preds = result.logits
#    print("PREDS SHAPE:", preds.shape)
    N_token = preds[:, pos, :]
#()    print("SHAPE", N_token.shape)
    return N_token

def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

def construct_whole_bert_embeddings(input_ids, ref_input_ids):
    
    input_embeddings = interpretable_embedding.indices_to_embeddings(input_ids)
    #, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = interpretable_embedding.indices_to_embeddings(ref_input_ids)
    #, token_type_ids=token_type_ids, position_ids=position_ids)

    return input_embeddings, ref_input_embeddings


def run_attribution_model(input_seq, ref_token_id, top_N_tokens, mask_index, layer):
    interpretable_embedding = configure_interpretable_embedding_layer(model, 'distilbert.embeddings')
    try:
        ablator = LayerIntegratedGradients(custom_forward, layer)

        input_tensor = torch.tensor([input_seq] * len(top_N_tokens))
        ref_tensor = torch.tensor([ref_token_id]).expand((1,len(input_seq)))
        interpretable_input_tensor = interpretable_embedding.indices_to_embeddings(input_tensor)
        ref_tensor = interpretable_embedding.indices_to_embeddings(ref_tensor)

        attention_mask = torch.ones_like(input_tensor)
        attributions = ablator.attribute(
                inputs=interpretable_input_tensor,
                baselines=ref_tensor,
                additional_forward_args=(attention_mask,mask_index),
                target=top_N_tokens,
        )
        attributions = summarize_attributions(attributions)
    finally:
        remove_interpretable_embedding_layer(model, interpretable_embedding)

    return attributions.T

In [5]:
def build_dataframe(attributions, input_seq, top_N_tokens):

    ix = pd.CategoricalIndex(
                [tokenizer.decode([token]) for token in input_seq],
                categories=reversed([tokenizer.decode([token]) for token in input_seq]),
                ordered=True
            )
#    print(ix)
    attr_df = (
        pd.DataFrame(
                attributions.detach().numpy(), 
                columns=[tokenizer.decode([token]) for token in top_N_tokens],
                index=ix,
            )
        .reset_index()
    )
    attr_df = attr_df.melt(id_vars=["index"])
    #attr_df = attr_df[~(attr_df['index'] == tokenizer.mask_token)]
    attr_df['variable'] = pd.Categorical(
            attr_df['variable'], 
            categories = [tokenizer.decode([token]) for token in top_N_tokens], 
            ordered=True
        )
    attr_df['display_value'] = attr_df['value'].apply(lambda f: f"{f:.2f}")
    return attr_df
    

In [6]:
def create_plot(attr_df, mask_index, N):
    ncol = np.ceil(np.sqrt(N))
    return (ggplot(attr_df, aes(x="index", y="value")) + 
        geom_col(aes(fill="index", colour="index"))  +
        geom_text(aes(y="value/2", label="display_value"), size=10) +
#        geom_label(aes(x = len(input_seq) - mask_index, y= 0, label="variable"), size=13, boxstyle="darrow") +
        scale_x_discrete(drop=False) +
        facet_wrap("~variable", ncol=ncol) +
        coord_flip() +
        labs(
                x="target token",
                y="Captum contribution scores",
#                title="Exploring the contribution of each token to the prediction."
            ) +
        theme(legend_position="none")
    )

In [7]:
def run_and_show(sentence, mask_tokens, reference=[tokenizer.unk_token_id], layer=None):
    if isinstance(mask_tokens, int):
        N = mask_tokens
        input_seq, mask_tokens, mask_index = prepare_data_for_sentence(sentence, N)
    else:
        N = len(mask_tokens)
        if isinstance(mask_tokens[0], str):
            mask_tokens = tokenizer.convert_tokens_to_ids(mask_tokens)
        input_seq, _, mask_index = prepare_data_for_sentence(sentence, N)

    attributions = []
    for ref in reference:
        attributions.append(run_attribution_model(input_seq, ref, mask_tokens, mask_index, layer))
    attributions = torch.stack(attributions).mean(axis=0)
    df = build_dataframe(attributions, input_seq, mask_tokens)
    plot = create_plot(df, mask_index, N)
    display(plot)
    return df, mask_index

In [8]:
def run_no_show(sentence, mask_tokens, reference=[tokenizer.unk_token_id], layer=None):
    if isinstance(mask_tokens, int):
        N = mask_tokens
        input_seq, mask_tokens, mask_index = prepare_data_for_sentence(sentence, N)
    else:
        N = len(mask_tokens)
        if isinstance(mask_tokens[0], str):
            mask_tokens = tokenizer.convert_tokens_to_ids(mask_tokens)
        input_seq, _, mask_index = prepare_data_for_sentence(sentence, N)

    attributions = []
    for ref in reference:
        attributions.append(run_attribution_model(input_seq, ref, mask_tokens, mask_index, layer))
    attributions = torch.stack(attributions).mean(axis=0)
    df = build_dataframe(attributions, input_seq, mask_tokens)
    return df, mask_index

In [170]:
from IPython.display import HTML
import re

def html_for_sentences(df, mask_index, groupings):
    if len(groupings) > 1:
        output = "<div>"
        group = groupings.pop(0)
        for g, gdf in df.groupby(group):
            output += f"<div>{group}: {g}</div><div style='padding-left:2em;'>"
            output += html_for_sentences(gdf, mask_index, groupings)
            output += "</div>"
        output += "</div>"
    else:
        output = "<table>"
        for label, lvl in df.groupby(groupings[0]):
            output += f"<tr><th>{label}</th>"
            for v, grp in lvl.groupby('variable'):
                for i, row in grp.iterrows():
                    pct = int(row['rel_value'] * 100)
                    g = 10 + pct
                    b = 150 + pct
                    if row['value'] >= 0:
                        r = g
                    else:
                        r = b
                    a = (row['rel_value'] / 2) + 0.3
                    color = ",".join(list(map(str, [r,g,b,a])))
                    size = (0.5 + (row['rel_value'] / 2) + 0.3) * 1.5

                    if i % len(grp) == mask_index:
                        output += f"<td><span style='font-size: {size}em; text-decoration:underline;  color: rgba({color}'>{row['variable']}</span></td>"
                    else:
                        word = row['index'].strip()
                        word = re.sub(r'<', '&lt;', re.sub(r'>', '&gt;', word))
                        output += f"<td><span style='font-size: {size}em; color: rgba({color});'>{word}</span></td>"
            output += "</tr><tr><th/>"
            for v, grp in lvl.groupby('variable'):
                for i, row in grp.iterrows():
                    output += f"<td style='font-size: 0.7em'>{round(row['value'],3)}</td>"
            output += "</tr>"
        output += "</table>"
    return output


In [156]:
from numpy.random import default_rng
rng = default_rng()
sample = rng.integers(model.config.vocab_size, size=20)
print(sample)

[18593 25119 18503  4291  7840 27513  8231  3386 23858 14233 20925  2194
 10027 14065  2072 16659 16280 19265 27845 12211]


In [157]:
print([tokenizer.decode([t]) for t in sample])

['artery', 'Telecom', '##bey', 'sector', 'fourteen', 'coordinating', '##EC', 'era', 'balancing', 'Waterloo', 'terminology', 'provide', 'offense', 'Audio', 'entire', '##claim', 'auditorium', 'Oracle', 'Evaluation', 'exam']


In [161]:
%%time
full_df = None
for sample in [[tokenizer.unk_token_id], [tokenizer.pad_token_id], sample]:
    sample_df = None
    for i in range(6):
        layer = getattr(model.distilbert.transformer.layer, f"{i}")
        df, mask_index = run_no_show(f"The cat from the neighbours chases a {tokenizer.mask_token}.", ["mouse"], reference=sample, layer=layer)
        df['layer'] = f"layer {i+1}"
        df['rel_value'] = (df['value'].abs() / df['value'].abs().max()).round(2)

        if sample_df is None:
            sample_df = df
        else:
            sample_df = pd.concat([sample_df, df])
    sample_df['sample'] = "; ".join([tokenizer.decode([t]) for t in sample])
    if full_df is None:
        full_df = sample_df
    else:
        full_df = pd.concat([full_df, sample_df])
        

CPU times: user 5min 55s, sys: 14.6 s, total: 6min 9s
Wall time: 1min 32s


In [172]:
html = html_for_sentences(full_df, mask_index, ["layer", "sample"])
display(HTML(html))

In [171]:
html = html_for_sentences(full_df, mask_index, ["sample", "layer"])
display(HTML(html))